In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
import requests
from tabulate import tabulate

<h3>Web Scraoing and Data Extraction</h3>

In [26]:
result= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(result,"lxml")
table1= soup.find("table",class_="wikitable")

In [27]:
n_rows=table1.find_all("tr")

In [34]:
neighbor=[]
for i in n_rows:
    outcome=i.text.split("\n")[1:-1]
    neighbor.append(outcome)

In [37]:
neighbor[:]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M5A', 'Downtown Toronto', 'Regent Park'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', 'Etobicoke', 'Islington Avenue'],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 [

<h3>Convertion of extracted data to Dataframe</h3>

In [39]:
neighbor[0][-1]="Neighborhood"
Ndf=pd.DataFrame(neighbor[1:],columns=neighbor[0])
Ndf.head(2)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned


In [47]:
B_na=Ndf.index[Ndf["Borough"]=="Not assigned"]
N_na=Ndf.index[Ndf["Neighborhood"]=="Not assigned"]
Gna=B_na & N_na

In [48]:
Ndf.drop(Ndf.index[B_na],inplace=True)
Ndf.reset_index(drop=True,inplace=True)

In [50]:
Ndf.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [53]:
postalCode_g=Ndf.groupby("Postcode")
N_group=postalCode_g['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
borough_g=postalCode_g['Borough'].apply(lambda x: set(x).pop())
df_group= pd.DataFrame(list(zip(borough_g.index,borough_g,N_group)))
df_group.columns=["Postcode", "Borough", "Neighborhood"]

<h3>using the .shape method to print the number of rows of your dataframe</h3>

In [55]:
df_group.shape

(103, 3)

<h3>getting the coordinates of the neighborhood</h3>

In [57]:
df_coordinates = pd.read_csv("Geospatial_Coordinates.csv")
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [62]:
pcode_coordinates=df_group.join(df_coordinates.set_index("Postal Code"),on="Postcode")
                                

In [63]:
pcode_coordinates.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
